This notebook prepares the data for the subsequent notebook `15-Step-Analyze.ipynb`, which generates the figure illustrating multi-step prediction performance against naive temporal baselines, as described in Supplementary Material Section 8.3.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
num_devices = torch.cuda.device_count()

import random
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from tqdm import tqdm
from data_processing import (
    SimpleSerializerSettings, scale_2d_array, unscale_2d_array,
    serialize_2d_integers, deserialize_2d_integers
)

from allen_cahn_equation import (
    compute_exact_solution_random_ic_vary_Nx,
    visualize_spline_ic,
)

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)


def train_ar_model(data_quantized, order):
    n_timesteps, n_spatial = data_quantized.shape
    X = []
    y = []
    for x_idx in range(n_spatial):
        time_series = data_quantized[:, x_idx]
        for t in range(order, n_timesteps):
            X.append(time_series[t-order:t][::-1])
            y.append(time_series[t])
    X = np.array(X)
    y = np.array(y)
    model = LinearRegression()
    model.fit(X, y)
    
    return model


def ar_multi_step_prediction(model, initial_data_quantized, order, n_steps):
    n_spatial = initial_data_quantized.shape[1]
    predictions = np.zeros((n_steps, n_spatial), dtype=int)
    buffer = initial_data_quantized[-order:].copy().astype(float)
    for step in range(n_steps):
        current_pred = np.zeros(n_spatial, dtype=int)
        for x_idx in range(n_spatial):
            features = buffer[:, x_idx][::-1].reshape(1, -1)
            pred_value = model.predict(features)[0]
            pred_value = int(round(pred_value))
            current_pred[x_idx] = pred_value
        predictions[step] = current_pred
        buffer = np.vstack([buffer[1:], current_pred])
    
    return predictions


def temporal_repeat_prediction(initial_data_quantized, n_steps):
    last_timestep = initial_data_quantized[-1]
    predictions = np.tile(last_timestep, (n_steps, 1))
    return predictions


def last_token_repeat_prediction(initial_data_quantized, n_steps):
    n_spatial = initial_data_quantized.shape[1]
    last_token_value = initial_data_quantized[-1, -1]
    predictions = np.full((n_steps, n_spatial), last_token_value, dtype=int)
    return predictions


def ar_predictions_evaluate(full_serialized_data, input_time_steps,
                                           number_of_future_predictions, ar_order,
                                           Nx, settings, vmin, vmax):
    u_quantized = deserialize_2d_integers(full_serialized_data, settings)
    
    u_training_quantized = u_quantized[:input_time_steps]
    u_ground_truth_quantized = u_quantized[input_time_steps:input_time_steps + number_of_future_predictions]
    model = train_ar_model(u_training_quantized, ar_order)
    predictions_quantized = ar_multi_step_prediction(
        model, u_training_quantized, ar_order, number_of_future_predictions
    )
    predictions_unscaled = unscale_2d_array(predictions_quantized, vmin, vmax)
    ground_truth_unscaled = unscale_2d_array(u_ground_truth_quantized, vmin, vmax)
    max_diffs = []
    rmses = []
    
    for step in range(number_of_future_predictions):
        pred_step = predictions_unscaled[step]
        true_step = ground_truth_unscaled[step]
        max_diff = np.max(np.abs(pred_step - true_step))
        rmse = np.sqrt(np.mean((pred_step - true_step)**2))
        max_diffs.append(max_diff)
        rmses.append(rmse)
    
    return np.array(max_diffs), np.array(rmses)


def temporal_repeat_predictions_evaluate(full_serialized_data, input_time_steps,
                                                  number_of_future_predictions,
                                                  Nx, settings, vmin, vmax):
    u_quantized = deserialize_2d_integers(full_serialized_data, settings)
    u_training_quantized = u_quantized[:input_time_steps]
    u_ground_truth_quantized = u_quantized[input_time_steps:input_time_steps + number_of_future_predictions]
    predictions_quantized = temporal_repeat_prediction(u_training_quantized, number_of_future_predictions)
    predictions_unscaled = unscale_2d_array(predictions_quantized, vmin, vmax)
    ground_truth_unscaled = unscale_2d_array(u_ground_truth_quantized, vmin, vmax)
    
    max_diffs = []
    rmses = []
    
    for step in range(number_of_future_predictions):
        pred_step = predictions_unscaled[step]
        true_step = ground_truth_unscaled[step]
        
        max_diff = np.max(np.abs(pred_step - true_step))
        rmse = np.sqrt(np.mean((pred_step - true_step)**2))
        
        max_diffs.append(max_diff)
        rmses.append(rmse)
    
    return np.array(max_diffs), np.array(rmses)


def last_token_repeat_predictions_evaluate(full_serialized_data, input_time_steps,
                                                    number_of_future_predictions,
                                                    Nx, settings, vmin, vmax):
    u_quantized = deserialize_2d_integers(full_serialized_data, settings)
    
    u_training_quantized = u_quantized[:input_time_steps]
    u_ground_truth_quantized = u_quantized[input_time_steps:input_time_steps + number_of_future_predictions]
    
    predictions_quantized = last_token_repeat_prediction(u_training_quantized, number_of_future_predictions)
    
    predictions_unscaled = unscale_2d_array(predictions_quantized, vmin, vmax)
    ground_truth_unscaled = unscale_2d_array(u_ground_truth_quantized, vmin, vmax)
    
    max_diffs = []
    rmses = []
    
    for step in range(number_of_future_predictions):
        pred_step = predictions_unscaled[step]
        true_step = ground_truth_unscaled[step]
        
        max_diff = np.max(np.abs(pred_step - true_step))
        rmse = np.sqrt(np.mean((pred_step - true_step)**2))
        
        max_diffs.append(max_diff)
        rmses.append(rmse)
    
    return np.array(max_diffs), np.array(rmses)

In [ ]:
# Define parameters for the Allen-Cahn equation
L = 2       # Length of the spatial domain
k = 0.001   # Thermal diffusivity
T = 1.0     # Total simulation time
Nt = 50     # Number of time steps
dt = T/Nt
Nx = 14     # Number of spatial steps (excluding boundary points)
dx = L/(Nx+1)
settings = SimpleSerializerSettings(space_sep=",", time_sep=";")
input_time_steps = 31
number_of_future_predictions = 15
n_ics = 20

# Generate all random initial conditions and spline objects
stored_initial_conditions = []
stored_spline_objects = []
for ic_seed in range(n_ics):
    random.seed(ic_seed)
    np.random.seed(ic_seed)
    torch.manual_seed(ic_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(ic_seed)
    init_cond_random = np.random.uniform(-0.5, 0.5, size=Nx)
    stored_initial_conditions.append(init_cond_random.copy())
    fig, cs = visualize_spline_ic(L, Nx, init_cond_random)
    plt.close(fig)
    stored_spline_objects.append(cs)

stored_initial_conditions_array = np.array(stored_initial_conditions)

# AR(1) model
all_ar_results = {'max_diffs': [], 'rmses': []}
for ic_seed in tqdm(range(n_ics), desc="AR(1)"):
    # Use the stored initial condition and spline
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    # Compute exact solution for this initial condition
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
    max_diffs, rmses = ar_predictions_evaluate(
        full_serialized_data=u_exact_serialized,
        input_time_steps=input_time_steps,
        number_of_future_predictions=number_of_future_predictions,
        ar_order=1,
        Nx=Nx,
        settings=settings,
        vmin=vmin_exact,
        vmax=vmax_exact
    )
    
    all_ar_results['max_diffs'].append(max_diffs)
    all_ar_results['rmses'].append(rmses)

# Temporal Repeat baseline
all_temporal_repeat_results = {'max_diffs': [], 'rmses': []}
for ic_seed in tqdm(range(n_ics), desc="Temporal Repeat"):
    # Use the stored initial condition and spline
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    # Compute exact solution for this initial condition
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
    max_diffs, rmses = temporal_repeat_predictions_evaluate(
        full_serialized_data=u_exact_serialized,
        input_time_steps=input_time_steps,
        number_of_future_predictions=number_of_future_predictions,
        Nx=Nx,
        settings=settings,
        vmin=vmin_exact,
        vmax=vmax_exact
    )
    all_temporal_repeat_results['max_diffs'].append(max_diffs)
    all_temporal_repeat_results['rmses'].append(rmses)

# Last-Token Repeat baseline
all_last_token_repeat_results = {'max_diffs': [], 'rmses': []}

for ic_seed in tqdm(range(n_ics), desc="Last-Token Repeat"):
    # Use the stored initial condition and spline
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    # Compute exact solution for this initial condition
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
    # Run last-token repeat baseline for this initial condition
    max_diffs, rmses = last_token_repeat_predictions_evaluate(
        full_serialized_data=u_exact_serialized,
        input_time_steps=input_time_steps,
        number_of_future_predictions=number_of_future_predictions,
        Nx=Nx,
        settings=settings,
        vmin=vmin_exact,
        vmax=vmax_exact
    )
    
    all_last_token_repeat_results['max_diffs'].append(max_diffs)
    all_last_token_repeat_results['rmses'].append(rmses)


def log_ci(mean, std, n, tcrit):
    se = std / np.sqrt(n)
    se_log = se / (mean * np.log(10))
    mean_log = np.log10(mean)
    delta_log = tcrit * se_log
    return 10**(mean_log - delta_log), 10**(mean_log + delta_log)

t_critical = stats.t.ppf(0.975, n_ics - 1)

averaged_results = {}

all_max_diffs = np.array(all_ar_results['max_diffs'])
all_rmses = np.array(all_ar_results['rmses'])

avg_max_diffs = np.mean(all_max_diffs, axis=0)
avg_rmses = np.mean(all_rmses, axis=0)
std_max_diffs = np.std(all_max_diffs, axis=0, ddof=1)
std_rmses = np.std(all_rmses, axis=0, ddof=1)

ci_lower_max_diffs = []
ci_upper_max_diffs = []
ci_lower_rmses = []
ci_upper_rmses = []

for mean, std in zip(avg_max_diffs, std_max_diffs):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_max_diffs.append(lower)
    ci_upper_max_diffs.append(upper)

for mean, std in zip(avg_rmses, std_rmses):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_rmses.append(lower)
    ci_upper_rmses.append(upper)

averaged_results['AR1'] = {
    'max_diffs': avg_max_diffs.tolist(),
    'rmses': avg_rmses.tolist(),
    'std_max_diffs': std_max_diffs.tolist(),
    'std_rmses': std_rmses.tolist(),
    'ci_lower_max_diffs': ci_lower_max_diffs,
    'ci_upper_max_diffs': ci_upper_max_diffs,
    'ci_lower_rmses': ci_lower_rmses,
    'ci_upper_rmses': ci_upper_rmses,
}

all_max_diffs = np.array(all_temporal_repeat_results['max_diffs'])
all_rmses = np.array(all_temporal_repeat_results['rmses'])

avg_max_diffs = np.mean(all_max_diffs, axis=0)
avg_rmses = np.mean(all_rmses, axis=0)
std_max_diffs = np.std(all_max_diffs, axis=0, ddof=1)
std_rmses = np.std(all_rmses, axis=0, ddof=1)

ci_lower_max_diffs = []
ci_upper_max_diffs = []
ci_lower_rmses = []
ci_upper_rmses = []

for mean, std in zip(avg_max_diffs, std_max_diffs):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_max_diffs.append(lower)
    ci_upper_max_diffs.append(upper)

for mean, std in zip(avg_rmses, std_rmses):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_rmses.append(lower)
    ci_upper_rmses.append(upper)

averaged_results['Temporal_Repeat'] = {
    'max_diffs': avg_max_diffs.tolist(),
    'rmses': avg_rmses.tolist(),
    'std_max_diffs': std_max_diffs.tolist(),
    'std_rmses': std_rmses.tolist(),
    'ci_lower_max_diffs': ci_lower_max_diffs,
    'ci_upper_max_diffs': ci_upper_max_diffs,
    'ci_lower_rmses': ci_lower_rmses,
    'ci_upper_rmses': ci_upper_rmses,
}

all_max_diffs = np.array(all_last_token_repeat_results['max_diffs'])
all_rmses = np.array(all_last_token_repeat_results['rmses'])

avg_max_diffs = np.mean(all_max_diffs, axis=0)
avg_rmses = np.mean(all_rmses, axis=0)
std_max_diffs = np.std(all_max_diffs, axis=0, ddof=1)
std_rmses = np.std(all_rmses, axis=0, ddof=1)

ci_lower_max_diffs = []
ci_upper_max_diffs = []
ci_lower_rmses = []
ci_upper_rmses = []

for mean, std in zip(avg_max_diffs, std_max_diffs):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_max_diffs.append(lower)
    ci_upper_max_diffs.append(upper)

for mean, std in zip(avg_rmses, std_rmses):
    lower, upper = log_ci(mean, std, n_ics, t_critical)
    ci_lower_rmses.append(lower)
    ci_upper_rmses.append(upper)

averaged_results['Last_Token_Repeat'] = {
    'max_diffs': avg_max_diffs.tolist(),
    'rmses': avg_rmses.tolist(),
    'std_max_diffs': std_max_diffs.tolist(),
    'std_rmses': std_rmses.tolist(),
    'ci_lower_max_diffs': ci_lower_max_diffs,
    'ci_upper_max_diffs': ci_upper_max_diffs,
    'ci_lower_rmses': ci_lower_rmses,
    'ci_upper_rmses': ci_upper_rmses,
}

all_baseline_max_errors_per_step = []
all_baseline_rmse_errors_per_step = []

for ic_seed in range(n_ics):
    init_cond_random = stored_initial_conditions[ic_seed]
    cs = stored_spline_objects[ic_seed]
    u_exact = compute_exact_solution_random_ic_vary_Nx(L, k, T, Nx, Nt, spline_obj=cs)
    
    u_exact_scaled, vmin_exact, vmax_exact = scale_2d_array(u_exact)
    u_exact_serialized = serialize_2d_integers(u_exact_scaled, settings)
    u_exact_parsed = deserialize_2d_integers(u_exact_serialized, settings)
    u_exact_unscaled = unscale_2d_array(u_exact_parsed, vmin_exact, vmax_exact)
    
    seed_max_errors_per_step = []
    seed_rmse_errors_per_step = []
    
    for t in range(u_exact.shape[0]):
        max_err_t = np.max(np.abs(u_exact[t] - u_exact_unscaled[t]))
        rmse_err_t = np.sqrt(np.mean((u_exact[t] - u_exact_unscaled[t])**2))
        seed_max_errors_per_step.append(max_err_t)
        seed_rmse_errors_per_step.append(rmse_err_t)
    
    all_baseline_max_errors_per_step.append(seed_max_errors_per_step)
    all_baseline_rmse_errors_per_step.append(seed_rmse_errors_per_step)

all_baseline_max_errors_per_step = np.array(all_baseline_max_errors_per_step)
all_baseline_rmse_errors_per_step = np.array(all_baseline_rmse_errors_per_step)

avg_baseline_max_errors_per_step = np.mean(all_baseline_max_errors_per_step, axis=0)
avg_baseline_rmse_errors_per_step = np.mean(all_baseline_rmse_errors_per_step, axis=0)

avg_baseline_max_errors_prediction = avg_baseline_max_errors_per_step[input_time_steps:]
avg_baseline_rmse_errors_prediction = avg_baseline_rmse_errors_per_step[input_time_steps:]

np.savez_compressed(
    "AR_Repeat_15_step.npz",
    ar_results=averaged_results,
    all_ar_results=all_ar_results,
    all_temporal_repeat_results=all_temporal_repeat_results,
    all_last_token_repeat_results=all_last_token_repeat_results,
    avg_baseline_max_errors_prediction=avg_baseline_max_errors_prediction,
    avg_baseline_rmse_errors_prediction=avg_baseline_rmse_errors_prediction,
    stored_initial_conditions=stored_initial_conditions_array,
)